# **Разведывательный анализ данных**

Перед тем как строить модель для кредитного скоринга, нам нужно провести разведывательный анализ данных, чтобы узнать больше о данных содержащихся в датасете.

Выделим несколько целей, которые мы бы хотели достичь в результате анализа данных:

1. Узнать о смысле признаков для нашей модели, посмотреть на их распределение, попопробовать на начальном этапе откинуть несколько заведамо неинформативных признаков, выделить целевую переменную;
2. Посмотреть, есть ли пропуски в нашем датасете, если же есть, то проанализировать стоит ли выкидывать объекты с пропусками, или заменить их какими-либо точечными статистиками (mean, median и т.д.);
3. Преобразовать признаки, если требуется их преобразовать;
4. Предложить варианты решения исходной задачи, построить baseline модели машинного обучения, выделить ключевые метрики для оптимизации модели. 


In [1]:
#Подключим библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use("seaborn-v0_8")

In [2]:
credit_score_df = pd.read_csv("~/Documents/datasets/train.csv")

' '.join(credit_score_df.columns)

/tmp/ipykernel_77842/3386353930.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  credit_score_df = pd.read_csv("~/Documents/datasets/train.csv")


'ID Customer_ID Month Name Age SSN Occupation Annual_Income Monthly_Inhand_Salary Num_Bank_Accounts Num_Credit_Card Interest_Rate Num_of_Loan Type_of_Loan Delay_from_due_date Num_of_Delayed_Payment Changed_Credit_Limit Num_Credit_Inquiries Credit_Mix Outstanding_Debt Credit_Utilization_Ratio Credit_History_Age Payment_of_Min_Amount Total_EMI_per_month Amount_invested_monthly Payment_Behaviour Monthly_Balance Credit_Score'

В датасете есть проблема с 27 столбцом, некоторые данные в столбце некорректны, большинство значений в столбце - это числа в формате `float`, проверим какие значения не преобразуется в `float` и удалим строки с данными значениями.

In [5]:
#Функция для проверки на преобразуемость в float
def is_convertible_to_float(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

bad_indexes = []

for index in range(len(credit_score_df["Monthly_Balance"])):
    val = credit_score_df["Monthly_Balance"][index]
    if not is_convertible_to_float(val):
        print(f'Value incorrect at index {index} and value {val}')
        bad_indexes.append(index)

credit_score_df.drop(index=bad_indexes, inplace = True)

print("Bad indexes deleted.")

Value incorrect at index 5545 and value __-333333333333333333333333333__
Value incorrect at index 26177 and value __-333333333333333333333333333__
Value incorrect at index 29158 and value __-333333333333333333333333333__
Value incorrect at index 35570 and value __-333333333333333333333333333__
Value incorrect at index 38622 and value __-333333333333333333333333333__
Value incorrect at index 60009 and value __-333333333333333333333333333__
Value incorrect at index 75251 and value __-333333333333333333333333333__
Value incorrect at index 82918 and value __-333333333333333333333333333__
Value incorrect at index 83255 and value __-333333333333333333333333333__
Bad indexes deleted.


In [7]:
#Заменим все значения на преобразованные в float
credit_score_df["Monthly_Balance"] = credit_score_df["Monthly_Balance"].astype(float)

Для начала нам нужно исследовать наш датасет [Credit score classification](https://www.kaggle.com/datasets/parisrohan/credit-score-classification), данный датасет взят с платформы Kaggle, датасет используется для задачи классификации кредитной истории (плохая, стандартная, хорошая), то есть для задачи кредитного скоринга, в задаче мы будем классифицировать только то плохая ли история у кредитора или же нет, то есть `Бинарную классификацию`.

Датасет содержит 28 признаков, приведём описание всех призаков содержащихся в датасете:

- `ID` - Уникальный идентификатор записи;
- `Customer_ID`- Уникальный идентификатор человека;
- `Month` - Месяц года в котором была сделана запись;
- `Name` - Имя человека;
- `Age` - Возраст человека;
- `SSN` - Номер социального страхования человека;
- `Occupation` - Род занятий человека;
- `Annual_Income` - Годовой доход лица;
- `Monthly_Inhand_Salary` - Месячная базовая зарплата человека;
- `Num_Bank_Accounts` - Количество банковских счетов, которыми владеет человек;
- `Num_Credit_Card` - Количество других кредитных карт, имеющихся у человека;
- `Interest_Rate` - Процентная ставка по кредитной карте;
- `Num_of_Loan` - Количество кредитов, взятых в банке;
- `Type_of_Loan` - Виды кредитов, которые берет человек;
- `Delay_from_due_date` - Среднее количество дней задержки с даты платежа;
- `Num_of_Delayed_Payment` - Среднее количество платежей, задержанных человеком;
- `Changed_Credit_Limit` - Процентное изменение лимита по кредитной карте;
- `Num_Credit_Inquiries` - Количество запросов по кредитным картам;
- `Credit_Mix` - Классификация сочетания кредитов;
- `Outstanding_Debt` - Оставшаяся задолженность к выплате (в долларах США);
- `Credit_Utilization_Ratio` - Коэффициент использования кредитной карты;
- `Credit_History_Age` - Возраст кредитной истории человека;
- `Payment_of_Min_Amount` - Отражает, была ли уплачена человеком только минимальная сумма;
- `Total_EMI_per_month` - Ежемесячные платежи EMI (в долларах США);
- `Amount_invested_monthly` - Ежемесячная сумма, инвестированная клиентом (в USD);
- `Payment_Behaviour` - Платежное поведение клиента (в USD);
- `Monthly_Balance` - Cумма ежемесячного баланса клиента (в USD);
- `Credit_Score` - Целевая переменная, категория кредитного рейтинга (Poor, Standard, Good).

Некоторые признаки в нашей выборке являются заведамо неприменимы с точки зрения модели машинного обучения, эти признаки - имена или какие либо идентификаторы: `ID`, `Customer_ID`, `Name`, `SSN`, удалим их. Также удалим признак который сложно анализировать, а именно `Type_of_Loan`, так как он содержит текстовые данные, для анализа данного признака нужна модель глубокого обучения для векторизации текстов, в силу ограниченности вычислительных ресурсов данный признак мы не будем исследовать.

In [9]:
#Удалим некачественные или сложные признаки
credit_score_df.drop(columns=['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan'], inplace = True)

Теперь для оставшихся признаков получим информацию об их типах данных.

In [11]:
credit_score_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99991 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     99991 non-null  object 
 1   Age                       99991 non-null  object 
 2   Occupation                99991 non-null  object 
 3   Annual_Income             99991 non-null  object 
 4   Monthly_Inhand_Salary     84990 non-null  float64
 5   Num_Bank_Accounts         99991 non-null  int64  
 6   Num_Credit_Card           99991 non-null  int64  
 7   Interest_Rate             99991 non-null  int64  
 8   Num_of_Loan               99991 non-null  object 
 9   Delay_from_due_date       99991 non-null  int64  
 10  Num_of_Delayed_Payment    92989 non-null  object 
 11  Changed_Credit_Limit      99991 non-null  object 
 12  Num_Credit_Inquiries      98026 non-null  float64
 13  Credit_Mix                99991 non-null  object 
 14  Outstanding

Как мы видим много признаков имеют тип данных `object`, хотя по смыслу они не должные являтся таковыми, внимательно посмотрим на все признаки которые являются категориальными и исправим их, если изначально они были числовыми, будем считать что если категориальный признак имеет имеет более 25 различных категорий - то этот признак должен являться числовым и по ошибке был преобразован в категориальный.

In [13]:
cat_features = credit_score_df.columns[(credit_score_df.dtypes == "object")]

bad_features = []

for feature in cat_features:
    cat_count = len(credit_score_df[feature].unique())
    if cat_count > 25:
        print(f'Feature {feature} have {cat_count} categories')
        bad_features.append(feature)

print(f"\nWe need to fix this features: {", ".join(bad_features)}")

Feature Age have 1788 categories
Feature Annual_Income have 18939 categories
Feature Num_of_Loan have 434 categories
Feature Num_of_Delayed_Payment have 750 categories
Feature Changed_Credit_Limit have 4384 categories
Feature Outstanding_Debt have 13178 categories
Feature Credit_History_Age have 405 categories
Feature Amount_invested_monthly have 91042 categories

We need to fix this features: Age, Annual_Income, Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Outstanding_Debt, Credit_History_Age, Amount_invested_monthly


Проверим вручную и преобразуем все строки с некорректными данными.

Как мы видим все некорректные данные в столбце `Age`, `Annual_Income`, `Num_of_Loan`, `Num_of_Delayed_Payment`, `Outstanding_Debt` имеют `_` на конце. В столбце `Changed_Credit_Limit` нужно заменить `_` на значение по смыслу, `Credit_History_Age` нужно перевести из формата времени в числа, а у `mount_invested_monthly` убрать лишние символы. Преобразуем все некорректные данные.

In [32]:
import re

@np.vectorize
def get_good_val(value, _type):
    type_map = {
        'int': int,
        'float': float,
    }
    if not is_convertible_to_float(value):
        to_return = value[:-1]
    else:
        to_return = value

    return type_map[_type](to_return)

@np.vectorize
def parse_age_to_months(age_str):
    if is_convertible_to_float(age_str):
        return age_str
    
    # Используем регулярное выражение для извлечения чисел
    match = re.match(r'(\d+)\s+Years?\s+and\s+(\d+)\s+Months?', age_str)
    if match:
        years = int(match.group(1))
        months = int(match.group(2))
        # Преобразуем годы в месяцы и складываем
        total_months = years * 12 + months
        return total_months
    else:
        raise ValueError("Input string is not in the expected format.")
    
#Первый тип плохих строк
credit_score_df['Age'] = get_good_val(credit_score_df['Age'], 'int')
credit_score_df['Annual_Income'] = get_good_val(credit_score_df['Annual_Income'], 'float')
credit_score_df['Num_of_Loan'] = get_good_val(credit_score_df['Num_of_Loan'], 'int')
credit_score_df['Num_of_Delayed_Payment'] = get_good_val(credit_score_df['Num_of_Delayed_Payment'], 'float')
credit_score_df['Outstanding_Debt'] = get_good_val(credit_score_df['Outstanding_Debt'], 'float')
#Второй тип 
credit_score_df.loc[credit_score_df['Changed_Credit_Limit'] == '_', 'Changed_Credit_Limit'] = 0
credit_score_df['Changed_Credit_Limit'] = credit_score_df['Changed_Credit_Limit'].astype(float)
#Третий тип
correct_values =  parse_age_to_months(credit_score_df.loc[~credit_score_df['Credit_History_Age'].isna(), 'Credit_History_Age'] )
credit_score_df.loc[~credit_score_df['Credit_History_Age'].isna(), 'Credit_History_Age'] = correct_values
credit_score_df['Credit_History_Age'] = credit_score_df['Credit_History_Age'].astype(float)
#Четвертый тип 
credit_score_df['Amount_invested_monthly'] = credit_score_df['Amount_invested_monthly'].replace('_', '', regex=True).astype(float)

Мы удалили все плохие категориальные признаки, теперь запустим ещё раз код для проверки наличия таких признаков.

In [44]:
cat_features = credit_score_df.columns[(credit_score_df.dtypes == "object")]

bad_features = []

for feature in cat_features:
    cat_count = len(credit_score_df[feature].unique())
    if cat_count > 25:
        print(f'Feature {feature} have {cat_count} categories')
        bad_features.append(feature)

print(f"\nWe need to fix this features: {", ".join(bad_features) if len(bad_features) > 0 else None}")


We need to fix this features: None


Отлично, теперь все проблемы с категориальными переменными исправлены. Теперь можно посчитать их количество.

In [55]:
print(f"Count of categorical features is {np.sum(credit_score_df.dtypes == 'object')}") 

Count of categorical features is 6


Теперь построим `pairplot` для каждой пары признаков типа `float`, чтобы визуально посмотреть как взаимодействуют друг с другом признаки. Но для начала закодируем целевую переменную как 0 - если значение `Poor` и 1 если значение `Standard` или `Good`.

In [ ]:
num_features = credit_score_df.columns[credit_score_df.dtypes != "object"]

cs_df_sub = credit_score_df[num_features]

sns.pairplot(cs_df_sub, hue = "TARGET")

In [ ]:
plt.show()

Посмотрим соотношения классов целевой переменной.

In [ ]:
from collections import Counter

classes = Counter(credit_score_df["TARGET"])

plt.bar(classes.keys(),classes.values(), edgecolor='black', alpha=0.7)

plt.title("Соотношение классов целевой переменной")

plt.xlabel("Классы")
plt.ylabel("Количество")

plt.show()

На графике виден явный дисбаланс классов. Поэтому при решении задачи нет смысла использовать метрику `accuracy`, значит нужно пытаться смотреть на разделение классов, будем максимизировать метрику `AUC ROC`. 

Попробуем найти закономерности в категориальных данных, которые качественно отражаются на значении целевой переменной.

In [ ]:
pd.crosstab(credit_score_df["CODE_GENDER"], credit_score_df["TARGET"], margins=True).style.background_gradient(cmap='winter')

Оценим значимость пола на целевую переменную. В кросс таблице видно, что женщин-кредиторов в 2 раза больше, чем мужчин, при этом соотношение людей которые не выплатили кредит сохраняется.

In [ ]:
for column in credit_score_df.columns[credit_score_df.dtypes == "object"]:
    sns.catplot(x = column, col = 'TARGET', kind = 'count', data = credit_score_df, stat='percent', aspect=1.5, sharey=False, edgecolor='black', alpha=0.7)
    plt.show()

Нет больших различий в графиках для разных классов целевой переменной.

Посмотрим на статистику признаков в датасете.

In [ ]:
credit_score_df.describe()

Построим таблицу парных корелляций для всех признаков.

In [ ]:
plt.figure(figsize = (15, 15))

sns.heatmap(credit_score_df.select_dtypes(include=['float64', 'int64']).corr(), cmap="winter", annot=True, fmt="0.2f")

plt.show()

Стоит избавиться от признаков имеющих слишком большую корреляцию с другими например `FLAG_EMP_PHONE`.

Исследуем датасет с помощью `ProfileReport` из библиотеки `pandas_profiling`.

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(credit_score_df, title="Отчет о профилировании данных", explorative=True)

profile.to_notebook_iframe()

Исходя из `ProfileReport` стоит удалить 12 повторяющихся строк, также нужно преобразовать признаки `OWN_CAR_AGE`, `OCCUPATION_TYPE`, `AMT_INCOME_TOTAL`. Также удалим признак `FLAG_MOBIL`- признак почти константный.

В качестве `baseline` модели для решения задачи классификации мы используем `Logistic regression`. После чего попробуем использовать `KNN`, `Random forest`, `GBM` из библиотеки `xgboost`. Подберем гипперпараметры для данных моделей. Также сделаем отбор признаков в датасете, посмотрим как улучшится качество. Также можно выделить признаки с помощью `PCA`.